In [1]:
from time import sleep
from doa_mts import *
o1 = doaMtsOverlay('doa_mts.bit')

In [2]:
# o1.mts_sync(adcTarget = -1, mixer_phase1=-96, mixer_phase2=-96, mixer_phase3=92, mixer_phase4=100)
# o1.mts_sync(adcTarget = -1, mixer_phase1=5, mixer_phase2=0, mixer_phase3=0, mixer_phase4=0)
# o1.mts_sync(adcTarget = -1, mixer_phase1=0, mixer_phase2=0, mixer_phase3=0, mixer_phase4=0)

In [3]:
o1.mts_sync()

In [4]:
o1.mts_sync(adcTarget = -1, mixer_phase1=-90, mixer_phase2=-90, mixer_phase3=90, mixer_phase4=90)
o1.mts_sync(adcTarget = -1, mixer_phase1=-90, mixer_phase2=-90, mixer_phase3=90, mixer_phase4=90)

In [5]:
o1.mts_sync(mixer_phase1=-24, mixer_phase2=-64.5, mixer_phase3=132.88, mixer_phase4=179.67)
o1.mts_sync(mixer_phase1=-24, mixer_phase2=-64.5, mixer_phase3=132.88, mixer_phase4=179.67)

In [9]:
o1.configure_adcs(nyquist_zone=1, centre_freq=325)

In [ ]:
import socket
import struct
import numpy as np
import netifaces as ni
from rfsoc4x2 import oled

iface = ni.gateways()['default'][ni.AF_INET][1]
ip_address = ni.ifaddresses(iface)[2][0]['addr']
oled = oled.oled_display()
oled.write("IP Addr({}):\n{}".format(iface, ip_address))


port = 4000
server_ip = ip_address  # Use the appropriate IP address or hostname
server_port = port  # Use a free port number

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the server IP and port
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((server_ip, server_port))

try:
    while True:
            server_socket.listen(1)

            # Accept a connection from the client
            client_socket, client_address = server_socket.accept()        
            # Receive and unpack the data size (number of Int16 elements) as a 16-bit integer
            data_size = struct.unpack('Q', client_socket.recv(64))[0]

            # Get data from ADCs
            data = o1.get_custom_data(data_size)
            # o1.dac0_player[:] = np.shape(data)
            # o1.dac1_player[:] = np.shape(data)
            # Pack the data as binary
            data_binary = struct.pack('h' * data_size, *data)

            # Send the data
            client_socket.sendall(data_binary)

except KeyboardInterrupt:
    print("Transmission interrupted by the user.")

In [8]:
freq = 5000
o1.adc_tiles[0].DynamicPLLConfig(1, 500, freq)
o1.adc_tiles[2].DynamicPLLConfig(1, 500, freq)

RuntimeError: Function XRFdc_DynamicPLLConfig call failed
stdout: metal: error:     
 ADC 0 timed out at state 15 in XRFdc_WaitForRestartClr


In [ ]:
data = o1.get_custom_data(data_size)

In [ ]:
iqData

In [ ]:
import numpy as np

def cphase(rawData):
    # Calibrates phases using FFT phase estimation
    # Takes the maximum of the abs value of the FFT of the signal
    # and calculates its angle, then calculates the right phase shift
    # with respect to the first rawData vector.

    # INPUTS:
    # rawData: 4 X n raw signal matrix

    # OUTPUTS:
    # fixed_rawData: 4 X n aligned signal matrix
    # weights: 4 X 1 weights vector for phase alignment

    max_val = np.zeros(4, dtype=np.cdouble)
    rawFt = np.zeros(len(rawData[:,1]), dtype=np.cdouble)
    angs = np.zeros(4, dtype=int)
    max_idx = np.zeros(4, dtype=int)
    for i in range(4):
        rawFt = np.fft.fftshift(np.fft.fft(rawData[i, :]))
        max_idx[i] = np.argmax(np.abs(rawFt))
        max_val[i] = rawFt[max_idx[i]]
        print(max_val[i])
    angs = np.angle(max_val)
    cal_angs = angs[0] - angs
    
    weights = np.exp(1j * cal_angs)
    
    return weights

In [ ]:
channels = 8 # 4 I + 4 Q
N = len(o1.adc_capture) // channels

# Data acq from FPGA
AlignedCaptureSamples = np.zeros((1,len(o1.adc_capture)),dtype=np.int16)
o1.internal_capture(AlignedCaptureSamples)

ShapedCaptureSamples = np.zeros((channels, N), dtype=np.int16)
for i in range(channels):
    ShapedCaptureSamples[i] = AlignedCaptureSamples[0][i::channels]

iqData = np.zeros((4, N), dtype=np.cdouble)
idx = 0
for i in range(0, 8, 2):
    iqData[idx] = np.conjugate(ShapedCaptureSamples[i] - 1j * ShapedCaptureSamples[i+1])
    idx = idx + 1
np.angle(cphase(iqData), deg=True)


In [ ]:
import matplotlib.pyplot as plt
for x in range(1, channels, 2):
    plt.plot(ShapedCaptureSamples[x][0:100])


plt.show()

In [ ]:
o1.mts_sync(mixer_phase1=-24, mixer_phase2=-64.5, mixer_phase3=132.88, mixer_phase4=179.67)
o1.mts_sync(mixer_phase1=-24, mixer_phase2=-64.5, mixer_phase3=132.88, mixer_phase4=179.67)


In [ ]:
o1.mts_sync(mixer_phase1=-90, mixer_phase2=-137.04, mixer_phase3=54.74, mixer_phase4=116.47)

In [ ]:
o1.mts_sync(mixer_phase1=-90)
 -103.07
     97.16
111.08

In [ ]:
o1.mts_sync(mixer_phase1=-90, mixer_phase2=-103.07, mixer_phase3=97.16, mixer_phase4=111.08)

In [ ]:
o1.mts_sync()

In [ ]:
array([  0.        , -40.19392807, -23.61730385,  23.4435106 ])

In [ ]:
o1.mts_sync(mixer_phase1=-90)
# o1.mts_sync()
channels = 8 # 4 I + 4 Q
N = len(o1.adc_capture) // channels

# Data acq from FPGA
AlignedCaptureSamples = np.zeros((1,len(o1.adc_capture)),dtype=np.int16)
o1.internal_capture(AlignedCaptureSamples)

ShapedCaptureSamples = np.zeros((channels, N), dtype=np.int16)
for i in range(channels):
    ShapedCaptureSamples[i] = AlignedCaptureSamples[0][i::channels]

iqData = np.zeros((4, N), dtype=np.cdouble)
idx = 0
for i in range(0, 8, 2):
    iqData[idx] = np.conjugate(ShapedCaptureSamples[i] - 1j * ShapedCaptureSamples[i+1])
    idx = idx + 1
phases = np.angle(cphase(iqData), deg=True)
print(phases)
o1.mts_sync(mixer_phase1=-90, mixer_phase2=phases[1], mixer_phase3=phases[2], mixer_phase4=phases[3])
import matplotlib.pyplot as plt
for x in range(1, channels, 2):
    plt.plot(ShapedCaptureSamples[x][0:100])


plt.show()

In [ ]:
channels = 8 # 4 I + 4 Q
N = len(o1.adc_capture) // channels

# Data acq from FPGA
AlignedCaptureSamples = np.zeros((1,len(o1.adc_capture)),dtype=np.int16)
o1.internal_capture(AlignedCaptureSamples)

ShapedCaptureSamples = np.zeros((channels, N), dtype=np.int16)
for i in range(channels):
    ShapedCaptureSamples[i] = AlignedCaptureSamples[0][i::channels]

iqData = np.zeros((4, N), dtype=np.cdouble)
idx = 0
for i in range(0, 8, 2):
    iqData[idx] = np.conjugate(ShapedCaptureSamples[i] - 1j * ShapedCaptureSamples[i+1])
    idx = idx + 1

import matplotlib.pyplot as plt
for x in range(1, channels, 2):
    plt.plot(ShapedCaptureSamples[x][0:100])


plt.show()

# 